## illustrating a simple algorithm of a pegged buy order and sell order

In [1]:
# Pegged order
# the use of pegged order is take advantage of the change in prices, spread
# so buy low sell high 
# For a pegged order to work the following infrmation is needed
# the tracking type, best bid, best ask and the buy offset and the sell offset, qty

# Say Now, if the best bid changes to $100.50, we would like to track this and buy if the price drops to the desired offset and sell for sell-offset

# Let us look at an example of pegged order. Suppose your strategy requires you 
# to buy a limit order to be filled at three ticks lower than the current best bid and a sell limit order to be filled at two ticks higher than the current best offer. 
# When the bid price changes, the pegged order becomes a composite order comprising

import threading
import time
import random

def set_timeout(func, seconds):
    timer = threading.Timer(seconds, func)
    timer.start()

def get_price_for_stock_type(type, stock):
    stock_data = {
        "AMZ": {
            "bid_prices": [100.50, 103.10, 102.30, 105.50],
            "ask_prices": [102.50, 105.10, 104.30, 107.50],
        },
    }
    if type not in ["bid", "ask"]:
        raise ValueError("Invalid order type. Use 'bid' or 'ask'.")
    else:
        return random.choice(stock_data[stock][f'{type}_prices'])

# internal_order_id_hash: { offer_id: #123, flip_offer_id: #321 }
placed_orders = {}

def pegged_order(stock, qty, type, offset, flip_offset):
    current_type_price = get_price_for_stock_type(type, stock)
    is_bid = type == 'bid'
    offer = current_type_price - offset if is_bid else current_type_price + offset
    flip_offer = current_type_price + flip_offset if is_bid else current_type_price - flip_offset
    internal_order_id = f'{stock}_{offer}_{flip_offer}'

    if internal_order_id in placed_orders:
        return
    else:
        placed_orders[internal_order_id] = {}
        place_offer_price = placed_orders[internal_order_id].get("offer", {}).get("price")
        place_flip_offer_price = placed_orders[internal_order_id].get("flip_offer", {}).get("price")
        offer_id_cancelled = False
        flip_offer_id_cancelled = False

        if place_offer_price != offer:
            offer_id_cancelled = cancel_order(placed_orders[internal_order_id].get("offer_id"))
        else:
            if place_offer_price == offer and order_status(placed_orders[internal_order_id].get("offer", {}).get("id")) == True:
                del placed_orders[internal_order_id]["offer"]
            placed_orders[internal_order_id]["offer"] = {"id": place_order(offer, qty), "price": offer}

        if place_flip_offer_price != flip_offer:
            flip_offer_id_cancelled = cancel_order(placed_orders[internal_order_id].get("flip_offer_id"))
        else:
            if place_offer_price == offer and order_status(placed_orders[internal_order_id].get("flip_offer", {}).get("id")) == True:
                del placed_orders[internal_order_id]["flip_offer"]
            placed_orders[internal_order_id]["flip_offer"] = {"id": place_order(flip_offer, qty), "price": flip_offer}

        if offer_id_cancelled and flip_offer_id_cancelled:
            del placed_orders[internal_order_id]

# This loop will check pegged orders every 1 seconds
while True:
    pegged_stocks = [{"stock": "AMZ", "qty": 10, "type": "bid", "offset": 3, "flip_offset": 2}]
    for peg in pegged_stocks:
        pegged_order(peg["stock"], peg["qty"], peg["type"], peg["offset"], peg["flip_offset"])
    
    time.sleep(1)  # Wait for 2 seconds before checking again


NameError: name 'cancel_order' is not defined